In [19]:
import json
import pandas as pd
from datetime import datetime
from airflow.decorators import dag, task
import os
import time
import pickle

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


In [20]:
csv_file_path = '/Users/renzhou/Downloads/Y3S2/IS3107/Data-Engine/ETL/01_Data_Collection/01_dataset/processed_data/filtered_df4.csv'
        
df = pd.read_csv(csv_file_path)

In [21]:
df.head()

,avg_storey_range,floor_area_sqm,normalized_resale_price,total_dwelling_units,vacancy,commercial,mrt_interchange,flat_model,town,age_of_flat
0,0.0000,0.116466,0.123296,0.311620,0.000000,1,0,improved,other,45
1,0.1250,0.293173,0.279669,0.165493,0.400000,0,0,model a,other,45
2,0.0625,0.236948,0.186714,0.144366,0.511111,0,0,model a,other,45
3,0.1875,0.116466,0.129280,0.309859,0.000000,0,0,improved,other,45
4,0.1250,0.056225,0.105921,0.309859,0.000000,0,0,improved,other,45


In [22]:
columns_of_interest = ['avg_storey_range', 'floor_area_sqm', 'normalized_resale_price',
            'total_dwelling_units', 'vacancy', 'commercial', 'mrt_interchange',
            'flat_model', 'town', 'age_of_flat']
            


In [23]:
filtered_features = ['floor_area_sqm', 'avg_storey_range', 'total_dwelling_units',
            'commercial', 'age_of_flat', 'flat_model_model a',
            'flat_model_new generation', 'flat_model_other',
            'flat_model_premium apartment', 'town_jurong west', 'town_other',
            'town_punggol', 'town_sengkang', 'town_tampines', 'town_woodlands',
            'town_yishun']
        


In [27]:
df = df[columns_of_interest]
df_ohe = pd.get_dummies(df, drop_first=True)
        
df_ohe.columns

diff = set(filtered_features) - set(df_ohe.columns)

print(diff)

for column in diff:
    df_ohe[column] = 0



{'town_yishun', 'town_other', 'town_punggol', 'town_jurong west', 'town_tampines', 'town_woodlands', 'flat_model_new generation', 'town_sengkang', 'flat_model_premium apartment'}


,floor_area_sqm,avg_storey_range,total_dwelling_units,commercial,age_of_flat,flat_model_model a,flat_model_new generation,flat_model_other,flat_model_premium apartment,town_jurong west,town_other,town_punggol,town_sengkang,town_tampines,town_woodlands,town_yishun
0,0.116466,0.0000,0.311620,1,45,0,0,0,0,0,0,0,0,0,0,0
1,0.293173,0.1250,0.165493,0,45,1,0,0,0,0,0,0,0,0,0,0
2,0.236948,0.0625,0.144366,0,45,1,0,0,0,0,0,0,0,0,0,0
3,0.116466,0.1875,0.309859,0,45,0,0,0,0,0,0,0,0,0,0,0
4,0.056225,0.1250,0.309859,0,45,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192482,0.148594,0.0000,0.065141,1,53,0,0,1,0,0,0,0,0,0,0,0
192483,0.236948,0.0625,0.244718,1,53,1,0,0,0,0,0,0,0,0,0,0
192484,0.200803,0.1250,0.265845,0,53,0,0,1,0,0,0,0,0,0,0,0
192485,0.325301,0.2500,0.197183,0,53,0,0,1,0,0,0,0,0,0,0,0


In [28]:

        
        
X = df_ohe[filtered_features]
y = df_ohe['normalized_resale_price']
        

In [29]:
lr = LinearRegression()
lr.fit(X, y)
knn = KNeighborsRegressor(n_neighbors=8, weights='distance')
knn.fit(X, y)

KNeighborsRegressor(n_neighbors=8, weights='distance')

In [30]:
def feature_engineering(download_path: str):
        csv_file_path = os.path.join(download_path, 'processed_data/filtered_df3.csv')
        
        df1 = pd.read_csv(csv_file_path)
        threshold = 0.05*len(df1)


        df1['month'] = pd.to_datetime(df1['month'])
        df1 = df1[df1['month'] > '2015-01-01']
        
        # use month and lease_commence_date to calculate the age of the flat
        df1['month'] = pd.to_datetime(df1['month'])
        df1['lease_commence_date'] = pd.to_datetime(df1['lease_commence_date'])

        df1.loc[:,'age_of_flat'] = df1['month'].dt.year - df1['lease_commence_date'].dt.year
        df1 = df1.sort_values('month')
        
        columns_of_interest = ['avg_storey_range', 'floor_area_sqm', 'normalized_resale_price',
       'total_dwelling_units', 'vacancy', 'commercial', 
       'mrt_interchange', 'flat_model', 'town', 'age_of_flat']
        
        df1 = df1[columns_of_interest]

        df1['flat_model'] = df1['flat_model'].str.lower()
        df1['town'] = df1['town'].str.lower()
        
        # find the flat_models that have less than 10,000
        other_models = df1['flat_model'].value_counts()[df1['flat_model'].value_counts() < threshold].index
        other_models = list(other_models)

        # replace the flat_models that have less than 10,000 with 'other'
        df1['flat_model'] = df1['flat_model'].replace(other_models, 'other')
        
        
        # find the flat_types that have less than 10,000
        other_models = df1['town'].value_counts()[df1['town'].value_counts() < threshold].index
        other_models = list(other_models)

        # replace the flat_models that have less than 10,000 with 'other'
        df1['town'] = df1['town'].replace(other_models, 'other')
        
        
        columns_to_be_scaled = ['avg_storey_range', 
       'floor_area_sqm',
       'normalized_resale_price', 
       'total_dwelling_units', 'vacancy']
        
        
        scaling_dic = {}

        for col in columns_to_be_scaled:
            col_min = min(df1[col])
            col_max = max(df1[col])
            df1[col] = df1[col].apply(lambda x: (x-col_min)/(col_max-col_min))
            scaling_dic[col] = (col_min, col_max)

        json_file_path = os.path.join(download_path, 'processed_data/scaling_info.json')

        # Store scaling_dic in JSON file
        with open(json_file_path, 'w') as json_file:
            json.dump(scaling_dic, json_file)
            
        newdf_file_path = os.path.join(download_path, 'processed_data/filtered_df4.csv')
        
        # Store df1 in csv file
        df1.to_csv(newdf_file_path, index=False)

        return download_path

In [31]:
feature_engineering("/Users/renzhou/Downloads/Y3S2/IS3107/Data-Engine/ETL/01_Data_Collection/01_dataset")

/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_5878/2679494023.py:4 DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

'/Users/renzhou/Downloads/Y3S2/IS3107/Data-Engine/ETL/01_Data_Collection/01_dataset'

In [32]:
def model_retrain(download_path: str):
        csv_file_path = os.path.join(download_path, 'processed_data/filtered_df4.csv')
        
        df = pd.read_csv(csv_file_path)
        
        columns_of_interest = ['avg_storey_range', 'floor_area_sqm', 'normalized_resale_price',
            'total_dwelling_units', 'vacancy', 'commercial', 'mrt_interchange',
            'flat_model', 'town', 'age_of_flat']
            
        filtered_features = ['floor_area_sqm', 'avg_storey_range', 'total_dwelling_units',
            'commercial', 'age_of_flat', 'flat_model_model a',
            'flat_model_new generation', 'flat_model_other',
            'flat_model_premium apartment', 'town_jurong west', 'town_other',
            'town_punggol', 'town_sengkang', 'town_tampines', 'town_woodlands',
            'town_yishun']
        df = df[columns_of_interest]
        df_ohe = pd.get_dummies(df, drop_first=True)
         
        diff = set(filtered_features) - set(df_ohe.columns)

        for column in diff:
            df_ohe[column] = 0
            
        X = df_ohe[filtered_features]
        y = df_ohe['normalized_resale_price']
        lr = LinearRegression()
        lr.fit(X, y)
        knn = KNeighborsRegressor(n_neighbors=8, weights='distance')
        knn.fit(X, y)
        # save models as pkl
        pkl_path_1 = os.path.join(download_path, 'processed_data/linear_model.pkl')
        pkl_path_2 = os.path.join(download_path, 'processed_data/knn_model.pkl')

        with open(pkl_path_1, 'wb') as f:
            pickle.dump(lr, f)
        with open(pkl_path_2, 'wb') as f:
            pickle.dump(knn, f)
            
        return download_path

In [33]:
model_retrain('/Users/renzhou/Downloads/Y3S2/IS3107/Data-Engine/ETL/01_Data_Collection/01_dataset')

'/Users/renzhou/Downloads/Y3S2/IS3107/Data-Engine/ETL/01_Data_Collection/01_dataset'